In [1]:
from pyiron_atomistics import Project

In [2]:
pr = Project('my_dev_stuff')
pr.remove_jobs(silently=True, recursive=True)

In [3]:
# Input
host = 'Fe'
x = 'Cr'
y = 'Co'
n = 3

In [4]:
# Build base structure
structure = pr.atomistics.structure.bulk(host).repeat(n)
structure[0] = x
structure[1] = y

In [5]:
# Get a potential
job = pr.atomistics.job.Lammps('dummy')
job.structure = structure
potential = job.list_potentials()[0]

In [6]:
def XY_energy(project, name, XY_structure, potential, host):
    job = project.atomistics.job.Lammps(name)
    job.structure = structure.copy()
    job.potential = potential
    job.calc_minimize()
    job.run()
    return job.output.energy_pot[-1]

In [7]:
def X_energy(project, name, XY_structure, potential, host):
    job = project.atomistics.job.Lammps(name)
    job.structure = XY_structure.copy()
    job.structure[1] = host  # Now X structure
    job.potential = potential
    job.calc_minimize()
    job.run()
    return job.output.energy_pot[-1]

In [8]:
def Y_energy(project, name, XY_structure, potential, host):
    job = project.atomistics.job.Lammps(name)
    job.structure = XY_structure.copy()
    job.structure[0] = host  # Now Y structure
    job.potential = potential
    job.calc_minimize()
    job.run()
    return job.output.energy_pot[-1]

In [9]:
def bulk_energy(project, name, XY_structure, potential, host):
    job = project.atomistics.job.Lammps(name)
    job.structure = XY_structure.copy()
    job.structure[0] = host
    job.structure[1] = host  # Now bulk structure
    job.potential = potential
    job.calc_minimize()
    job.run()
    return job.output.energy_pot[-1]

In [10]:
E_xy = XY_energy(pr, 'xy', structure, potential, host)
E_x = X_energy(pr, 'x', structure, potential, host)
E_y = Y_energy(pr, 'y', structure, potential, host)
E_bulk = bulk_energy(pr, 'bulk', structure, potential, host)

The job xy was saved and received the ID: 166
The job x was saved and received the ID: 167
The job y was saved and received the ID: 168
The job bulk was saved and received the ID: 169


In [11]:
E_bind = (E_xy + E_bulk) - (E_x + E_y)
print(f"{E_bind:.4f} eV")

0.0336 eV
